<h1 style="text-align:center;">Mito Miner <span style="text-align:center;font-size: 0.5em;">0.2.1</span></h1>
<h2 style="text-align:center;">CeMiA Toolkit <i style="font-size: 0.5em;">0.6.0</i></h2>
<h3 style="text-align:center;">Kashatus Lab @ UVA</h3>

# Welcome to Mito Miner
#### Mito Miner is part of CeMiA toolkit that enables you to segment and extract the mitochondrial network of the cells from fluorescently labeled single-cell images.

This Jupyter notebook provides you with step-by-step directions to segment the mitochondrial network in your cells.

## 0) Import necessary libraries
Please check the requirements in the Readme file.

##### Just run the following block of code, you are not expected to enter any data here.

In [1]:
#Base Libraries
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.gridspec import GridSpec
import cv2

#Interaction
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider, Checkbox, FloatSlider, Dropdown
from IPython.display import clear_output

#Core Functions
import cemia55s as cemia

layout = widgets.Layout(width='90%')
style = {'description_width': 'initial'}

## 1) Choose your plan
Choose your plan here.

In [2]:
plan = widgets.Dropdown(
    options=['I am using Mito Miner to segment the cells that I previously isolated by Cell Catcher.', 'I have my own set of single cell images.'],
    value='I am using Mito Miner to segment the cells that I previously isolated by Cell Catcher.',
    description='Your Plan:',
    layout=layout,style=style)
display(plan)

Dropdown(description='Your Plan:', layout=Layout(width='90%'), options=('I am using Mito Miner to segment the …

## 2) Locate and sample files

<br>
<details>
<summary><span style="font-size:16px;font-weight: bold; color:red">What should I do next? What should I enter in the input box of the next cell? (click here to expand)</span></summary>
    
#### <span style="color:red;">You need to interact with the next cell: </span>  Please run the next cell, a box will appear. Enter the relative/absolute address of the folder that contains your images, then press enter.  (read the notes below). 

#### <span style="color:red;">Examples: </span>
* Relative Address
    * Use . if the images are in the same folder as this file
    * If your folder of the images (my_folder_of_images) is located in the same directory as this file, you should enter: my_folder_of_images
* Absolute Address
    *If your images are located on Desktop
        * Mac: you should enter: /Users/username/Desktop/my_folder_of_images
        * Windows: you should enter: C:\Users\username\Desktop\my_folder_of_images

#### <span style="color:red;">Notes: </span>
* It is preferred to have the folder of your images in the same folder as the current file that you are running
* After you entered the desired value in the box, you should press the enter key.
* If you have previously used Cell Catcher to isolate individual cells, and you want to analyze those images, you should enter the address where your original images (Multi-cell images) are located. This is the same address you used in Cell Catcher.
    * Example: If you have previously used Cell Catcher to isolate individual cells in a folder named "test", you should enter "test" as the address to your files. 
        * Cell Catcher has already created and filled the appropriate set of subfolders in the "test" directory, and Mito Miner will locate the files automatically.
* If you want to use Mito Miner with your own single cell images, you should enter the address where your files are located, and Mito Miner will guide you how to relocate your files.
    
</details>

In [3]:
address,file_list = cemia.address()

Where are the files located?

test

All set! You may run the next cell.


## 3) Satisfy basic requirements based on your plan.

In [8]:
cemia.check_dir_requirements_miner(address)

Test1: Directory requirements already satisfied


In [9]:
counter, temp_address = cemia.check_requirements_miner(plan.value,address)

Your Plan:
"I am using Mito Miner to segment the cells that I previously isolated by Cell Catcher."

We found 15 images based on your plan.


In [10]:
cemia.confirm_requirements_miner(plan.value, address, counter)


We found 15 images based on your plan. It seems your are all set!

You are all set!


## 4) Segment Sample Cells

### 4-1) Selecting a filtering method

<br>
<details>
<summary><span style="font-size:16px;font-weight: bold; color:red">Additional info</span></summary>

##### There are two different method available to help you to segment your mitochondrial network. You can test them and then decide which one works for your dataset better.
#### <span style="color:red;">Availabe Methods: </span>
#### <b>Let It Go:<b> Better option for images with very low background intensity.
#### <b>Not One of Us:</b> Signal separation based on the distribution of background intensity in the nucleus area.
#### <span style="color:red;">Notes: </span>

###### Once you selected a method, go and run the next cell. Rerunning the same cell may reset your selection.
</details>

In [11]:
filter_method = widgets.Dropdown(options=['Let It Go','Not One of Us'],description='Filter:',style=style, layout=layout)
print('\nLet It Go: Better option for images with low background intensity.')
print('\nNot One of Us: Signal separation based on the distribution of background intensity in the nucleus area.')
display(filter_method)


Let It Go: Better option for images with low background intensity.

Not One of Us: Signal separation based on the distribution of background intensity in the nucleus area.


Dropdown(description='Filter:', layout=Layout(width='90%'), options=('Let It Go', 'Not One of Us'), style=Desc…

### 4-2) Tweaking the selected filter parameters for accurate segmentation

<br>
<details>
<summary><span style="font-size:16px;font-weight: bold; color:red">List of parameters you are adjusting here (click to expand the list)</span></summary>
    
<p><b>Dark Image:</b> This option only visually enhances the dark images and has no effect on the analysis. Selecting this option makes images 3X brighter, and it is useful for comparison when signal level is low, and it is hard to visualize the original cells.</p>
<p><b>Filter Strength:</b> Determines the strength of the background removal.</p>
<p><b>Filter Harshness:</b> Determines if the background removal is harshly applied to a small group of images, or whether it is more softly applied to a wider range of images.</p>
<p></p>
<p><b>Adaptive Options</b></p>
<p>Settings for the adaptive method are effective only if the adaptive method is selected.</p>
<p><b>Adaptive Filter Tile Size:</b> The size of the tile over which the adpative function samples data and compares with the reference intensity.</p>
<p><b>Adaptive Filter Power:</b> The strength of the adaptive function in correcting the threshold.</p>
<p><b>Adaptive Filter Footprint:</b> Determines whther the correction factor should affect a wide, or short range of of images with different background intenisties.</p>
<p><b>Remove Debris:</b> Tries to remove noisy objects created in the image as a result of using different filters</p>

</details>

In [15]:
#Initializing model parameters
params = cemia.mito_miner_initial_params

tmp1 = []
tmp2 = []
sample_files = cemia.check_files_before_filter(temp_address)
sample_files = cemia.random_files(temp_address,sample_files,10)

print('Selected Method: ',filter_method.value)

if filter_method.value=='Not One of Us':
    @interact(dark=Checkbox(value=False,description='Dark Image',layout = layout, style=style),
              thresh9=IntSlider(value=2, min=1, max=5,step=1, description='Filter Harshness:',continuous_update=False,layout = layout, style=style),
              thresh10=IntSlider(value=1, min=1, max=10,step=1, description='Filter Strength:',continuous_update=False,layout = layout, style=style),
              adapt=Checkbox(value=False,description='Adaptive Filter Threshold',layout = layout, style=style),
              window=Dropdown(options=[8,16,32,64,128,256,512], value=16, description='Adaptive Filter Tile Size',layout = layout, style=style),
              equalizer=FloatSlider(value=0.15, min=0.1, max=0.5,step=0.05, description='Adaptive Filter Power:',continuous_update=False,layout = layout, style=style),
              steepness=FloatSlider(value=0.5, min=0.25, max=1.5,step=0.25, description='Adaptive Filter Footprint:',continuous_update=False,layout = layout, style=style),
              debries=Checkbox(value=True,description='Remove Debris',layout = layout, style=style))
    
    def filter_selection3(dark,thresh10, thresh9,adapt,window, equalizer,steepness, debries):
        
        #Updating parameters based on user interaction
        params['thresh_median_mask'].append(thresh9)
        params['thresh_median_mask2'].append(thresh10)
        params['adaptive_th'].append(adapt)
        params['window_size_th'].append(window)
        params['steepness_th'].append(steepness)
        params['equalizer_th'].append(equalizer)
        params['remove_debries_th'].append(debries)

        for file in sample_files:
            try:
                pic2 = plt.imread(os.path.join(temp_address, file))

                print('Strength: ', thresh10)
                print('Harshness: ', thresh9)
                print('File: ',os.path.join(temp_address, file))

                _,_, binarized, blue_mask = cemia.behind_the_moon_filter(pic2, 60,2.5,1,bg_harshness=-0.5,sig_harshness=params['thresh_median_mask'][-1],sig_strength=params['thresh_median_mask2'][-1], cleanup=False, method=2, adaptive=params['adaptive_th'][-1], window_size=params['window_size_th'][-1],equalizer=params['equalizer_th'][-1],steepness=params['steepness_th'][-1], remove_debries=params['remove_debries_th'][-1])
                params['out'].append(binarized)
                cemia.filter_plot_mito_miner(pic2,binarized,dark)

            except:
                print('\n{} is not a valid image file, trying the next file.'.format(file))
                pass
                        
elif filter_method.value=='Let It Go':
    @interact(dark=Checkbox(value=False,description='Dark Image',layout = layout, style=style),
              thresh11=FloatSlider(value=-0.5, min=-1.5, max=1,step=0.1, description='Filter Harshness:',continuous_update=False,layout = layout, style=style),
              adapt=Checkbox(value=False,description='Adaptive Filter Threshold',layout = layout, style=style),
              window=Dropdown(options=[8,16,32,64,128,256,512], value=16, description='Adaptive Filter Tile Size',layout = layout, style=style),
              equalizer=FloatSlider(value=0.15, min=0.1, max=0.5,step=0.05, description='Adaptive Filter Power',continuous_update=False,layout = layout, style=style),
              steepness=FloatSlider(value=0.5, min=0.25, max=1.5,step=0.25, description='Adaptive Filter Footprint',continuous_update=False,layout = layout, style=style),
              debries=Checkbox(value=True,description='Remove Debries',layout = layout, style=style))        
    
    def filter_selection3(dark,thresh11,adapt,window, equalizer,steepness, debries):

        params['thresh_median_signal'].append(thresh11)
        params['adaptive_th'].append(adapt)
        params['window_size_th'].append(window)
        params['steepness_th'].append(steepness)
        params['equalizer_th'].append(equalizer)
        params['remove_debries_th'].append(debries)

        for file in sample_files:
           
            try:
                pic2 = plt.imread(os.path.join(temp_address, file))

                print('Harshness: ', thresh11)
                print('File: ',os.path.join(temp_address, file))
                
                _,_,binarized, blue_mask = cemia.behind_the_moon_filter(pic2, 60,2.5,1,bg_harshness=params['thresh_median_signal'][-1],sig_harshness=2,sig_strength=1, cleanup=False, method=1, adaptive=params['adaptive_th'][-1], window_size=params['window_size_th'][-1],equalizer=params['equalizer_th'][-1],steepness=params['steepness_th'][-1], remove_debries=params['remove_debries_th'][-1])
                params['out'].append(binarized)
                cemia.filter_plot_mito_miner(pic2,binarized,dark)

            except:
                print('\n{} is not a valid image file, trying the next file.'.format(file))
                pass         


These files were validated and selected:

 ['2-19-new_cell20_1024.tif', '2-19-new_cell15_1024.tif', '2-19-new_cell10_1024.tif', '2-19-new_cell16_1024.tif', '2-19-new_cell8_1024.tif', '2-19-new_cell9_1024.tif', '2-19-new_cell21_1024.tif', '2-19-new_cell5_1024.tif', '2-19-new_cell4_1024.tif', '2-19-new_cell12_1024.tif']
Selected Method:  Not One of Us


interactive(children=(Checkbox(value=False, description='Dark Image', layout=Layout(width='90%'), style=Descri…

## Export all the settings
##### This part generates a file that should be copied to the same folder, where the images reside.

In [ ]:
print('The following settings will be used to analyze all the images!')
print('**************************************************************')
params_pd = {}
for k in params:
    if k != 'out':
        try:
            params_pd[k] = params[k][-1]
            #print(f'{k}: {params_pd[k]}')
        except:
            pass
    
params_pd = pd.DataFrame(params_pd, index=[0])
params_pd.to_csv(os.path.join(address ,'mito_miner_params.csv'), index=False)

cemia.report_mito_miner(filter_method.value,params)

## 4) Processing all the cells

In [14]:
final_folder = os.path.join(address, 'output','processed','single_cells_binary')
try:
    os.makedirs(final_folder)
except FileExistsError:
    pass

cell_list = os.listdir(temp_address)

if filter_method.value=='Not One of Us':
  
        for file in cell_list:
            fullpath_input = file
            abspath = address
                         
            try:
                pic2 = plt.imread(os.path.join(temp_address, file))
                print('\n\nProcessing File: ',os.path.join(temp_address, file))

                img,_,binarized, blue_mask = cemia.behind_the_moon_filter(pic2, 60,2.5,1,bg_harshness=-0.5,sig_harshness=params['thresh_median_mask'][-1],sig_strength=params['thresh_median_mask2'][-1], cleanup=False, method=2, adaptive=params['adaptive_th'][-1], window_size=params['window_size_th'][-1],equalizer=params['equalizer_th'][-1],steepness=params['steepness_th'][-1], remove_debries=params['remove_debries_th'][-1])
                cv2.imwrite(final_folder + '/' + file[:file.rfind('.')]+'_mask.tif', binarized)

                cv2.imwrite(final_folder + '/' + file, cv2.cvtColor(pic2, cv2.COLOR_RGB2BGR))

            except:
                print('\n{} is not a valid image file, trying the next file.'.format(file))
                pass

elif filter_method.value=='Let It Go':

    for file in cell_list:
        fullpath_input = file
        abspath = address

        try:
            pic2 = plt.imread(os.path.join(temp_address, file))

            print('\n\nProcessing File: ',os.path.join(temp_address, file))

            img,_,binarized, blue_mask = cemia.behind_the_moon_filter(pic2, 60,2.5,1,bg_harshness=params['thresh_median_signal'][-1],sig_harshness=2,sig_strength=1, cleanup=False, method=1, adaptive=params['adaptive_th'][-1], window_size=params['window_size_th'][-1],equalizer=params['equalizer_th'][-1],steepness=params['steepness_th'][-1], remove_debries=params['remove_debries_th'][-1])
            cv2.imwrite(final_folder + '/' + file[:file.rfind('.')]+'_mask.tif', binarized)
            cv2.imwrite(final_folder + '/' + file, cv2.cvtColor(pic2, cv2.COLOR_RGB2BGR))

        except:
            print('\n{} is not a valid image file, trying the next file.'.format(file))
            pass

print('\nDone with the filtering!\n')
print('\nYou are all set!')




Processing File:  test/output/to_analyze/2-19-new_cell19_1024.tif

Average background:  16.857639163047768


Processing File:  test/output/to_analyze/2-19-new_cell12_1024.tif

Average background:  15.021106863885658


Processing File:  test/output/to_analyze/2-19-new_cell21_1024.tif

Average background:  17.302174618630314


Processing File:  test/output/to_analyze/2-19-new_cell20_1024.tif

Average background:  17.509107893507707


Processing File:  test/output/to_analyze/2-19-new_cell15_1024.tif

Average background:  22.438533129601332


Processing File:  test/output/to_analyze/2-19-new_cell14_1024.tif

Average background:  14.775560734051849


Processing File:  test/output/to_analyze/2-19-new_cell6_1024.tif

Average background:  21.25967302452316


Processing File:  test/output/to_analyze/2-19-new_cell7_1024.tif

Average background:  18.741799617102743


Processing File:  test/output/to_analyze/2-19-new_cell10_1024.tif

Average background:  18.88173190984579


Processing File:  tes

# The End!